In [1]:
import pandas as pd
import numpy as py
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [2]:
x_data = pd.DataFrame(pd.read_csv('D:\jupyter\Methylation.csv',header=0,index_col=66)) 

c:\users\86300\pycharmprojects\aml\venv\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#以病人的id作为列索引.从第2行开始读取
x_id = x_data.iloc[1:]
#x_data.dtypes

In [4]:
#转置
x_reshape=x_id.T

In [5]:
x_reshape.shape

(66, 663827)

In [6]:
x_reshape.head()

,PTPRS,ITPK1,JARID2,JARID2,NBPF20,NBPF20,NBPF20,MORF4L1,TBC1D24,OR8G1,...,EMP2,BANP,TBCD,RNPS1,CHST6,SLC38A2,6-Mar,SYNJ1,NTN5,NTN5
047,0.939358638,0.030046451,0.018400715,0.018400715,0.97252672,0.97252672,0.97252672,0.725564722,0.900509671,0.981647151,...,0.968172,0.902462,0.910461,0.733044,0.315276,0.00950484,0.00968838,0.749168,0.963387,0.963387
048,0.932434087,0.016749009,0.012394616,0.012394616,0.947042586,0.947042586,0.947042586,0.359462468,0.913342585,0.976811811,...,0.964142,0.789957,0.88849,0.732056,0.414313,0.0063113,0.0066306,0.79246,0.952832,0.952832
049,0.948112616,0.023318525,0.01303935,0.01303935,0.970545077,0.970545077,0.970545077,0.664358528,0.817598465,0.953283771,...,0.971285,0.770995,0.884876,0.723788,0.398355,0.00541983,0.00748246,0.810468,0.972101,0.972101
050,0.939991017,0.019263045,0.015816588,0.015816588,0.967688823,0.967688823,0.967688823,0.587711466,0.838871016,0.916976496,...,0.968642,0.936802,0.892965,0.733599,0.426442,0.00244892,0.00867456,0.747421,0.94709,0.94709
014,0.862317569,0.024858933,0.02114024,0.02114024,0.971281178,0.971281178,0.971281178,0.600037305,0.859663984,0.937143282,...,0.955842,0.930599,0.904201,0.743525,0.358386,0.00577477,0.0160836,0.852471,0.958893,0.958893


In [7]:
#x_reshape.info

In [9]:
y_data = pd.DataFrame(pd.read_csv('./drug_sensity_num.csv',usecols=[4,5]))
#y_data = pd.DataFrame(pd.read_csv('D:\jupyter\drug_sensity_num.csv'))

In [10]:
y_data.head()

,sensity,name
0,高度敏感,014 杨立柯
1,高度敏感,014 杨立柯
2,高度敏感,014 杨立柯
3,不敏感,014 杨立柯
4,低度敏感,014 杨立柯


In [11]:
#取特定药物下病例的药敏数据
y_target = y_data.iloc[0::24]

In [12]:
y_target.shape

(54, 2)

In [13]:
#y_target.info()

In [14]:
def fun(x):
    if(x=='高度敏感'):
        return 3
    if(x=='中度敏感'):
        return 2
    if(x=='低度敏感'):
        return 1
    else:
        return 0

def delete_name(x):
    lists = x.split(' ')
    return lists[0]

In [15]:
#敏感度(sensity) 中文-数值 替换
y_target['sensity'] = y_target['sensity'].apply(lambda x: fun(x))

c:\users\86300\pycharmprojects\aml\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
#只保留病例的id，去掉名字
y_target['name'] = y_target['name'].apply(lambda x: delete_name(x))

c:\users\86300\pycharmprojects\aml\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
y_target.set_index(["name"],inplace=True)
#y_target

In [18]:
#表合并
total_data = x_reshape.join(y_target)

In [19]:
#处理空值
data_cleaned = total_data.fillna(value=0)

In [20]:
data_cleaned.shape

(66, 663828)

In [21]:
data = data_cleaned.iloc[:,0:663827]

In [22]:
#设置训练集&测试集
Xtrain, Xtest, Ytrain, Ytest = train_test_split(data, data_cleaned['sensity'], test_size=0.4) 

In [23]:
clf = DecisionTreeClassifier(
                            #criterion="entropy"
                             #,random_state=50
                            )
#rfc = RandomForestClassifier(random_state=0)

clf = clf.fit(Xtrain, Ytrain)
#rfc = rfc.fit(Xtrain, Ytrain)

score_c = clf.score(Xtest, Ytest)
#score_r = rfc.score(Xtest, Ytest)

In [24]:
#print("single Tree:{}".format(score_c), "Random Forest:{}".format(score_r))
print("single Tree:{}".format(score_c))

single Tree:0.37037037037037035


In [28]:
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [29]:
#rfc = RandomForestClassifier(n_estimators=25)
#rfc_s = cross_val_score(rfc, data, data_cleaned['sensity'], cv=10)
#clf = DecisionTreeClassifier()
#clf_s = cross_val_score(clf, data, data_cleaned['sensity'], cv=10)

In [30]:
#plt.plot(range(1,11),rfc_s,label="Randomforest")
#plt.plot(range(1,11),clf_s,label="Decision Tree")
#plt.legend()
#plt.show()